In [3]:
from settings import *


## BPIC2012
caseid = 'Case ID'
activity = 'Activity'
ts = 'Complete Timestamp'
label = {'Activity' : 'O_ACCEPTED-COMPLETE'}
other_features = ['Resource', 'Variant index', '(case) AMOUNT_REQ']

## BPIC2017
# caseid = 'Case ID'
# activity = 'Activity'
# ts = 'Complete Timestamp'
# label = {'column' : 'Accepted'}
# other_features = ['Resource', 'CreditScore', 'FirstWithdrawalAmount', 'MonthlyCost', 'NumberOfTerms','OfferedAmount']

## BPIC2015
# caseid = 'Case ID'
# activity = 'Activity'
# ts = 'Complete Timestamp'
# label = {'column' : 'Label'}
# other_features = ['Resource', 'monitoringResource', '(case) Includes_subCases','(case) Responsible_actor','(case) caseProcedure','(case) caseStatus','(case) last_phase','(case) parts',
#                 '(case) requestComplete','(case) termName', '(case) SUMleges']

combi = ['bucketing', 'encoding', 'drop_act', 'params']

options = {
    'bucketing' : (1,40), # a number of partitions
    
    'encoding' : ['index', 'aggregate'],
    
    'drop_act' : [2,4,6,8], # a number of activities to drop
    
    'models' : ['Decision Tree','Random Forest','LightGBM','Xgboost'],

    'params' : {'Decision Tree':{'max_depth': (2,20),
                           'min_samples_leaf': (5,100),
                           'criterion': ["gini", "entropy"]
            }, 
            'Random Forest':{"n_estimators": (10,1000), 
                           "max_depth": (2,20),
                           "max_features": ["auto", "log2"], 
                           "bootstrap": [True, False],
                           "criterion": ["gini", "entropy"]
            },
            'LightGBM':{'max_depth': (2,20),
                      'num_leaves' : (10,500),
                      'min_child_samples' : (2,10)
            },
            'Xgboost':{"max_depth": (2,20),
                     "n_estimators": (10,1000),
                     "learning_rate": [0.01, 0.05, 0.1]
                     
            }
            }
}

In [4]:
from tqdm import tqdm
import pandas as pd
import pickle

def add_label(df):       
    Label = []
    if 'Activity' in label:
        label_col = label['Activity']                
        for case, group in df.groupby(caseid):
            for i in range(len(group)):
                if label_col in group[activity].tolist():
                    Label.append(1)
                else:
                    Label.append(0)
        label_df = pd.DataFrame(Label, columns = ['Label'])
        df = pd.concat([df, label_df], axis=1)

    elif 'column' in label:
        label_col = label['column']
        df = df.rename(columns={label_col : 'Label'})
    return df

def drop_activity(df, n):
    trace_num = df['Case ID'].nunique()
    act = df['Activity'].value_counts()
    df = df.iloc[[i for i in range(len(df)) if df.iloc[i]['Activity'] not in act[-n:]]]
    return df

def whole_bucket(df):
    result = []
    
    for prefix in tqdm(range(2,42)):
        bucket=[]
        for case, group in df.groupby(caseid):
            group = group.sort_values(by=ts, ascending = True).reset_index(drop=True)
            if len(group) >= prefix:
                bucket.append(group.iloc[:prefix,:])
        new_df = pd.concat(bucket)
        result.append(new_df)
    
    return result

def prefix_bound(m, drop_low=False):
    if m == 1:
        if drop_low == True:
            return [[4,40]]
        else:
            return [[2,40]]
    
    if drop_low == True:
        if m > 37:
            m = 37
        prefix_len = 37//m
        remain = 37%m
        prev = 4
        bound=[]
        for i in range(m):
            if i < remain:
                bound.append([prev,prev+prefix_len+1])
                prev = prev+prefix_len+1
            else:
                bound.append([prev,prev+prefix_len])
                prev = prev+prefix_len
    else:  
        prefix_len = 39//m
        remain = 39%m
        prev = 2
        bound=[]
        for i in range(m):
            if i < remain:
                bound.append([prev,prev+prefix_len+1])
                prev = prev+prefix_len+1
            else:
                bound.append([prev,prev+prefix_len])
                prev = prev+prefix_len
        
    return bound

In [5]:
"""
GA : Genetic Algorithm
RS : Random Search
"""
df = pd.read_csv('/Users/yeon1\GA_based_AutoML\dataset\dataset\BPIC12.csv')
df = add_label(df)

In [5]:
# df1 = drop_activity(df, 2)
# df2 = drop_activity(df, 4)
# df3 = drop_activity(df, 6)
# df4 = drop_activity(df, 8)

# df_list1 = whole_bucket(df1)
# df_list2 = whole_bucket(df2)
# df_list3 = whole_bucket(df3)
# df_list4 = whole_bucket(df4)

# with open('df_list1.pkl', 'wb') as f1:
#     pickle.dump(df_list1, f1)
    
# with open('df_list2.pkl', 'wb') as f2:
#     pickle.dump(df_list2, f2)
    
# with open('df_list3.pkl', 'wb') as f3:
#     pickle.dump(df_list3, f3)
    
# with open('df_list4.pkl', 'wb') as f4:
#     pickle.dump(df_list4, f4)

In [6]:
from settings import *

In [7]:
import pandas as pd
import numpy as np
import re
import pickle
import time
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import permutations
import json, os
from collections import OrderedDict
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, precision_score, recall_score
from sklearn.preprocessing import OneHotEncoder


from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

class Genetic_Algorithm:
    def __init__(self, df, options : dict, combi : list , caseid : str, activity : str, ts : str, label : dict, other_features : list, phase):
        self.df = df
        self.options = options
        self.combi = combi
        self.caseid = caseid
        self.activity = activity 
        self.ts = ts
        self.label = label
        self.other_features = other_features
        
             
    def feature_type(self):
        df = self.df
        feature_dict = {}
        for feature in self.other_features:
            for case, group in df.groupby(self.caseid):
                if len(group[feature].unique()) != 1:
                    feature_dict[feature] = 'event'
                    break
                else:
                    feature_dict[feature] = 'case'
        self.feature_types = feature_dict
        
    def add_label(self, df):       
        Label = []
        if 'Activity' in self.label:
            label_col = self.label['Activity']                
            for case, group in df.groupby(self.caseid):
                for i in range(len(group)):
                    if label_col in group[self.activity].tolist():
                        Label.append(1)
                    else:
                        Label.append(0)
            label_df = pd.DataFrame(Label, columns = ['Label'])
            df = pd.concat([df, label_df], axis=1)

        elif 'column' in self.label:
            label_col = self.label['column']
            df = df.rename(columns={label_col : 'Label'})
        self.df = df
        return df
                

    def rand_choice(self, options, key : str):
        if  type(options[key]) == tuple:
            return np.random.randint(options[key][0], options[key][1])

        elif type(options[key]) == list:
            return options[key][np.random.randint(0, len(options[key]))]
               
        
    def initial_populations(self, N : int, rand_state = 2022) -> dict:
        initial_pop = []
        if 'params' in self.combi:
            for model in list(self.options['params'].keys()):
                for n in range(N):
                    result = {}
                    result['bucketing'] = self.rand_choice(self.options, 'bucketing')
                    result['encoding'] = self.rand_choice(self.options, 'encoding')
                    result['drop_act'] = self.rand_choice(self.options, 'drop_act')
                    result['models'] = model
                    result[model] = {}
                    for hp in list(self.options['params'][model].keys()):                             
                        result[model][hp] = self.rand_choice(self.options['params'][model], hp)
                    initial_pop.append(result)
        else:
            for n in range(N):
                result = {}
                result['bucketing'] = rand_choice(self.options, 'bucketing')
                result['encoding'] = rand_choice(self.options, 'encoding')
                result['drop_act'] = rand_choice(self.options, 'drop_act')
                initial_pop.append(result)                
        
        self.population = initial_pop
        
        return initial_pop
    
    def select_population(self, population, fitness, N) -> list:
#         population = self.population
        sum_fit = sum(fitness)
        selection_probs = [fitness[c]/sum_fit for c in range(len(population))]

        return list(np.random.choice(population, int(N), p=selection_probs))

    
    # Roullette wheel selection
    def select_param(self, population, fitness, hp : bool) -> list: 
#         population = self.population
        sum_fit = sum(fitness)
        selection_probs = [fitness[c]/sum_fit for c in range(len(fitness))]

        if hp == False:
            return np.random.choice(population, 2, p=selection_probs)

        else:
            p1 = np.random.choice(population, p=selection_probs)
            hp_space = [c for c in range(len(population)) if list(p1.keys())[-1] in list(population[c].keys())]
            hp_sum_fit = sum([fitness[c] for c in hp_space])
            hp_selection_probs = [fitness[c]/hp_sum_fit for c in hp_space]
            p2 = population[np.random.choice(hp_space, p=hp_selection_probs)]
            return [p1, p2]
        
        
    # cp : crossover probability ~ (0,1) -> 0.9
    def crossover(self, population, fitness, num_offering : int, cp : float) -> dict:
#         population = self.population
        result = []
        n = int(num_offering*cp)
        hp_options = [True, False]
        for _ in range(n):
            hp_option = hp_options[np.random.randint(0, 2)]
            child = {}
            p1, p2 = self.select_param(population, fitness, hp=hp_option)
            co_point = np.random.randint(low=0, high=len(p1))
            for idx, key in enumerate(list(p2.keys())):
                if idx < co_point:
                    child[key] = p1[key]
                else:
                    child[key] = p2[key]
            result.append(child)
#         self.population = population.extend(result)
        
        return result
    

    # mp : crossover probability ~ (0,1) -> 0.03
    def mutation(self, population, N, mp : float):
#         population = self.population
        
        n = int(N*mp)

        result = []
        for _ in range(n):
            child = {}
            parent = population[np.random.choice(len(population))]
            params = [list(parent[key].keys())+[key] if key in list(self.options['params'].keys()) 
                      else key for key in list(parent.keys())]
            params.extend(params.pop())

            ml_model = params[-1]
            mut_param = params[np.random.randint(low=0, high=len(params)-2)]
            if mut_param in list(self.options.keys()):
                parent[mut_param] = self.rand_choice(self.options, mut_param)
            else:
                parent[ml_model][mut_param] = self.rand_choice(self.options['params'][ml_model], mut_param)
            result.append(parent)

        return result

    
    
    def fitness(self, tot_score, failure_rate, tot_time):
        highest_acc_pop = np.argmax(tot_score)

        # fitness = (acc + (1-failure_rate) + time_cost + acc_decrease)/4
        tot_fitness = [round((tot_score[i] + (1-failure_rate[i]) + (max(tot_time)-tot_time[i])/(max(tot_time)-min(tot_time)) 
                   + (tot_score[i]-min(tot_score))/(max(tot_score)-min(tot_score)))/4, 2) for i in range(len(tot_score))]
        
        return tot_fitness
    
        
    def indexbased_encoding(self, df, prefix):
        #df = self.df
        new_df = pd.DataFrame()
        for column in df.columns:
            if (column == 'Label') or (column == self.caseid) or ((column in self.feature_types) and (self.feature_types[column] == 'case')):
                case_df = df.groupby(self.caseid, as_index = False).first()[column]
                new_df = pd.concat([new_df, case_df], axis=1)
            elif (column == self.activity) or ((column in self.feature_types) and (self.feature_types[column] == 'event')):
                col_name = [str(column)+str(i+1) for i in range(prefix)]
                col_list = []
                for case, group in df.groupby(self.caseid):
                    group = group.sort_values(by=self.ts, ascending = True).reset_index(drop=True)
                    col_list.append(group[column].tolist())
                col_list = np.array(col_list)
                event_df = pd.DataFrame(col_list, columns = col_name)
                new_df = pd.concat([new_df, event_df], axis=1)

        self.df = new_df
        return self.one_hot_encoding(new_df)
        
    def aggregated_encoding(self, df):
        # df = self.df
        new_df = pd.DataFrame()
        for column in df.columns:       
            if (column == 'Label') or (column == self.caseid) or ((column in self.feature_types) and (self.feature_types[column] == 'case')):
                case_df = df.groupby(self.caseid, as_index = False).first()[column]
                new_df = pd.concat([new_df, case_df], axis=1)
            elif (column == self.activity) or ((column in self.feature_types) and (self.feature_types[column] == 'case')):
                result = []
                if df[column].dtype == int or df[column].dtype == float:
                    if (column in self.feature_types) and (self.feature_types[column] == 'case'):
                        col_name = column
                        case_df = df.groupby(self.caseid, as_index = False).first()
                        not_nan = [num for num in list(case_df[column]) if num != np.nan]
                        fir_point, sec_point = np.percentile(not_nan,[33,67])

                        for val in case_df[column].values:
                            if val < fir_point:
                                result.append('Low')
                            elif fir_point <= val < sec_point:
                                result.append('Medium')
                            elif val >= sec_point:
                                result.append('High')
                            else:
                                result.append('Nan')
                    else:
                        col_name = [str(column)+'-'+point for point in ['Low', 'Medium', 'High', 'Nan']]
                        not_nan = [num for num in list(df[column]) if num != np.nan]
                        fir_point, sec_point = np.percentile(not_nan,[33,67])
                        for case, group in df.groupby(self.caseid):
                            col_list = [0]*len(col_name)
                            for val in group[column].values:
                                if val < fir_point:
                                    col_list[0] += 1
                                elif fir_point <= val < sec_point:
                                    col_list[1] += 1
                                elif val >= sec_point:
                                    col_list[2] += 1
                                else:
                                    col_list[3] += 1
                            result.append(col_list)
                                    
                else:
                    col_name = df[column].unique()

                    for case, group in df.groupby(self.caseid):
                        group = group.sort_values(by=self.ts, ascending = True).reset_index(drop=True)
                        col_list = [0]*len(col_name)
                        key = list(group[column].value_counts().keys())
                        val = group[column].value_counts().values
                        for k in key:
                            col_list[key.index(k)] += val[key.index(k)]
                        result.append(col_list)

                result = np.array(result)
                event_df = pd.DataFrame(result, columns = col_name)
                new_df = pd.concat([new_df, event_df], axis=1)
        self.df = new_df
        return new_df

    
    def last_state_encoding(self, df, window):
        # df = self.df        
        event = {}
        caseid = []
        next_event = []
        for k in range(window):
            dict_index = 'event{}'.format(k+1)
            event[dict_index] = []
            for case, group in df.groupby(self.caseid):
                df1 = list(group[self.activity])
                L = len(df1) - window + k
                for j in range(k,L):
                    event[dict_index].append(df1[j])
                    if k == window -1:
                        caseid.append(case)
                        next_event.append(df1[j+1])
        
        df_1 = pd.DataFrame(caseid, columns = [self.caseid])
        df_2 = pd.DataFrame.from_dict(event)
        df_3 = pd.DataFrame(next_event, columns = ['Label'])
        new_df = pd.concat([df_1, df_2, df_3], axis=1)        
        self.df = new_df
        
        return new_df
    
    
    def encoding(self, df, key, prefix):
        if key == "index":
            return self.indexbased_encoding(df, prefix)
        
        elif key == "aggregate":
            return self.aggregated_encoding(df)
        
        else:
            return self.last_state_encoding(df, 3)
    
    def one_hot_encoding(self, df):
        #df = self.df
        for column in df.columns:
            if not np.issubdtype(df[column], np.number):
                one_hot = pd.get_dummies(df[column], prefix=column, prefix_sep='=')
                #print("Encoded column:{} - Different keys: {}".format(column, one_hot.shape[1]))
                df = df.drop(column, axis=1)
                df = df.join(one_hot)
        #print("Categorical columns encoded")
        self.df = df
        return df
     
    
    def train_test_set_split(self, df, encoding):
                
        df_train, df_test = train_test_split(df, test_size=0.2, random_state=1, shuffle=False)
#         print(f'Training samples: {len(df_train)} \nTest samples: {len(df_test)}')
        X_train = df_train.drop('Label', axis=1)
        y_train = df_train['Label']
        X_test = df_test.drop('Label', axis=1)
        y_test = df_test['Label']
        
        if encoding == 'last_state':
            ohe = OneHotEncoder(handle_unknown='ignore',sparse=False)
            ohe.fit(X_train.values)
            X_train = ohe.transform(X_train.values)
            X_test = ohe.transform(X_test.values)

            #X_train = self.one_hot_encoding(X_train)
            #X_test = self.one_hot_encoding(X_test)
        
        else:
            ratio = len(df_train[df_train['Label'] == 1]) / len(df_train[df_train['Label'] == 0])     
#             print(f'Ratio of target in training set 0 : 1 = 1:{ratio:.4f}')

            # For imbalanced data
            if ratio < 0.33:
                sm = SMOTE(random_state=0)
                sm_X_train, sm_y_train = sm.fit_resample(X_train, y_train)
                print('After OverSampling, the shape of train_X: {}'.format(sm_X_train.shape))
                print('After OverSampling, the shape of train_y: {} \n'.format(sm_y_train.shape))
                print("After OverSampling, counts of label '1': {}".format(sum([sm_y_train[i]==1 for i in range(len(sm_y_train))])))
                print("After OverSampling, counts of label '0': {}".format(sum([sm_y_train[i]==0 for i in range(len(sm_y_train))])))
                X_train = sm_X_train
                y_train = sm_y_train
                            
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        
        return X_train, y_train, X_test, y_test
     
    def model_fit(self):
        #['Logistic Regression','Decision Tree','Random Forest','LightGBM','Xgboost','CatBoost']
        models = {'Decision Tree' : DecisionTreeClassifier(), 'Random Forest' : RandomForestClassifier(), 'LightGBM' : LGBMClassifier(), 'Xgboost' : XGBClassifier()}
        tot_score = []
        for model in models:
            score = []

            model.fit(self.X_train, self.y_train)

            score.append(accuracy_score(self.y_test, model.predict(self.X_test)))
            score.append(precision_score(self.y_test, model.predict(self.X_test)))
            score.append(recall_score(self.y_test, model.predict(self.X_test)))
            score.append(f1_score(self.y_test, model.predict(self.X_test)))
            score.append(roc_auc_score(self.y_test, model.predict_proba(self.X_test)[:, 1]))

            print(f'Model is {model} \nAccuracy: {score[0]:.4f}, Precision: {score[1]:.4f}, Recall: {score[2]:.4f}, F1: {score[3]:.4f}, AUC: {score[4]:.4f}')
            tot_score.append(score)

        self.tot_score = tot_score

        # plot score df
        score_df = pd.DataFrame(tot_score, index = models, columns = ['Accuracy', 'Precision', 'Recall', 'F1 score', 'AUC'])
        score_df.plot(kind="bar",figsize=(9,8))
        plt.xticks(rotation='horizontal')
        plt.show()

        self.score_df = score_df
        return
    
    def decision_tree(self, hp, X_train, y_train, X_test, y_test):
        model = DecisionTreeClassifier(max_depth = hp['max_depth'],
                           min_samples_leaf= hp['min_samples_leaf'],
                           criterion = hp['criterion'])
        model.fit(X_train, y_train)
        score  = round((accuracy_score(y_test, model.predict(X_test))+roc_auc_score(y_test, model.predict_proba(X_test)[:, 1]))/2, 4)
        return score
    
    def random_forest(self, hp, X_train, y_train, X_test, y_test):
        model = RandomForestClassifier(n_estimators=hp['n_estimators'], 
                           max_depth= hp["max_depth"],
                           max_features= hp["max_features"], 
                           bootstrap= hp["bootstrap"],
                           criterion= hp["criterion"])
        model.fit(X_train, y_train)
        score  = round((accuracy_score(y_test, model.predict(X_test))+roc_auc_score(y_test, model.predict_proba(X_test)[:, 1]))/2, 4)
        return score
    
    def light_gbm(self, hp, X_train, y_train, X_test, y_test):
        model = LGBMClassifier(max_depth= hp["max_depth"],
                           num_leaves= hp["num_leaves"], 
                           min_child_samples= hp["min_child_samples"])
        model.fit(X_train, y_train)
        score  = round((accuracy_score(y_test, model.predict(X_test))+roc_auc_score(y_test, model.predict_proba(X_test)[:, 1]))/2, 4)
        return score
    
    def xgboost(self, hp, X_train, y_train, X_test, y_test):
        model = XGBClassifier(max_depth = hp["max_depth"],
                           n_estimators = hp["n_estimators"], 
                           learning_rate = hp["learning_rate"])
        model.fit(X_train, y_train)
        score  = round((accuracy_score(y_test, model.predict(X_test))+roc_auc_score(y_test, model.predict_proba(X_test)[:, 1]))/2, 4)
        return score
        
    def select_best_model(self):
        score_df = self.score_df
        arg_index = np.argmax(score_df['Accuracy'] + score_df['AUC'])
        print(f'Best model is {score_df.index[arg_index]} \nAccuracy: {score_df.iloc[arg_index][0]:.4f}, Precision: {score_df.iloc[arg_index][1]:.4f}, Recall: {score_df.iloc[arg_index][2]:.4f}, F1: {score_df.iloc[arg_index][3]:.4f}, AUC: {score_df.iloc[arg_index][4]:.4f}')
    
    
    def main(self):
        
        populations = self.initial_populations(20)
        tot_score = []
        result_df, prefix_idx = whole_bucket(df)
        for pop in populations:
            df1 = drop_activity(df, pop['drop_act'])
            bucket_list = bucketing(result_df, prefix_idx, pop['bucketing'])
            for bucket in bucket_list:
                score = []
                df1 = result_df[bucket[0]:bucket[1]]
                df1 = self.encoding(df1, pop['encoding'])
                X_train, y_train, X_test, y_test = self.train_test_set_split(df1, pop['encoding'])
                if 'Decision Tree' in pop:
                    score.append(self.decision_tree(pop['Decision Tree'], X_train, y_train, X_test, y_test))
                elif 'Random Forest' in pop:
                    score.append(self.random_forest(pop['Random Forest'], X_train, y_train, X_test, y_test))
                elif 'LightGBM' in pop:
                    score.append(self.light_gbm(pop['LightGBM'], X_train, y_train, X_test, y_test))
                else: 
                    score.append(self.xgboost(pop['Xgboost'], X_train, y_train, X_test, y_test))
                tot_score.append(score)
        print(tot_score)        
        new_df, prefix_idx = whole_bucket(df)


In [8]:
df = pd.read_csv('/Users/yeon1\GA_based_AutoML\dataset\dataset\BPIC12.csv')

In [9]:
# Create these file in 'drop_act+prefix_extraction.jpynb'

with open('./df_list1.pkl', 'rb') as f:
    df_list1 = pickle.load(f)
    
with open('./df_list2.pkl', 'rb') as f:
    df_list2 = pickle.load(f)
    
with open('./df_list3.pkl', 'rb') as f:
    df_list3 = pickle.load(f)
    
with open('./df_list4.pkl', 'rb') as f:
    df_list4 = pickle.load(f)

# Experiment 1

In [10]:
#selection rate
sr = 0.1
#crossover rate
cr = 0.9
#mutation rate
mr = 0.01

max_iter = 20

In [11]:
"""
GA : Genetic Algorithm
RS : Random Search
"""
phase = 1 # drop_activity(GA) + encoding(GA) + bucketing(GA) + ML params(GA) + hyperparameter opt(GA)
# phase = 2 # drop_activity(GA) + encoding(GA) + bucketing(GA) + ML params(GA) + hyperparameter opt(RS)
# phase = 3 # drop_activity(GA) + encoding(GA) + bucketing(GA) + ML params(RS) + hyperparameter opt(RS)
# phase = 4 # drop_activity(RS) + encoding(RS) + bucketing(RS) + ML params(RS) + hyperparameter opt(RS)

GA = Genetic_Algorithm(df, options, combi, caseid, activity, ts, label, other_features, phase)
GA.feature_type()

In [12]:
cnt = 0

# #selection rate
# sr_param = [0.1, 0.2, 0.3]
# #crossover rate
# cr_param = [0.9, 0.8, 0.7]
# #mutation rate
# mr_param = [0.1, 0.05, 0.01]

best_pop = []
# param_test = {'fitness':[], 'time':[], 'score':[], 'failure rate':[]}

populations = GA.initial_populations(5)

# for i in range(3):
#     sr = sr_param[i]
#     cr = cr_param[i]
#     for j in range(3):
#         print('trial =',i+j+1)
#         mr = mr_param[j]
generation_fitness = []
generation_score = []
generation_time = []
generation_failure_rate = []
for n_iter in range(max_iter):
    new_population = []
    tot_score = []
    tot_time = []
    failure_rate = []
    for pop in tqdm(populations):
        start_time = time.time()
        new_population.append(pop)
        print(pop)
        if pop['drop_act'] == 2:
            df_list = df_list1
        elif pop['drop_act'] == 4:
            df_list = df_list2
        elif pop['drop_act'] == 6:
            df_list = df_list3
        else:
            df_list = df_list4

        if pop['encoding'] == 'last_state':
            if pop['bucketing'] < 5:
                pop['bucketing'] = 5
            bound_list = prefix_bound(pop['bucketing'], drop_low=True)
        else:
            bound_list = prefix_bound(pop['bucketing'])

        score = []
        for bounds in bound_list:
            lower, upper = bounds
            merge_df = pd.DataFrame()
            print('bucket : ',(lower, upper))
            for idx in range(lower, upper):
                prefix_df = df_list[idx-2]
                prefix_df = GA.encoding(prefix_df, pop['encoding'], idx)
                merge_df = pd.concat([merge_df, prefix_df], sort=False)

            merge_df = merge_df.fillna(0)

            X_train, y_train, X_test, y_test = GA.train_test_set_split(merge_df, pop['encoding'])    

            #start = time.time()
            if 'Decision Tree' in pop:
                score.append(GA.decision_tree(pop['Decision Tree'], X_train, y_train, X_test, y_test))
            elif 'Random Forest' in pop:
                score.append(GA.random_forest(pop['Random Forest'], X_train, y_train, X_test, y_test))
            elif 'LightGBM' in pop:
                score.append(GA.light_gbm(pop['LightGBM'], X_train, y_train, X_test, y_test))
            else: 
                score.append(GA.xgboost(pop['Xgboost'], X_train, y_train, X_test, y_test))                    

        end_time = time.time()

        elapsed_time = round(end_time-start_time,2)

        tot_time.append(elapsed_time)
        tot_score.append(round(sum(score)/len(score),2))

        min_proba = 0.7
        failure_rate.append(len([i for i in score if i < min_proba])/len(score))

    fitness = GA.fitness(tot_score, failure_rate, tot_time)
    best_pop.append(populations[np.argmax(fitness)])

    N = len(new_population)
    pop1 = GA.select_population(new_population, fitness, N*sr)
    pop2 = GA.crossover(new_population, fitness, N, cr)
    pop3 = GA.mutation(new_population, N, mr)

    populations = pop1 + pop2 + pop3

    generation_fitness.append(round(sum(fitness)/len(fitness),4))
    generation_time.append(sum(tot_time))
    generation_score.append(round(sum(tot_score)/len(tot_score),4))
    generation_failure_rate.append(round(sum(failure_rate)/len(failure_rate),4))
    
    if cnt > 5:
        break
    
    elif len(generation_fitness) > 1:
        if abs(generation_fitness[-1]-generation_fitness[-2]) < 0.001:
            break 
        elif (generation_fitness[-1]-generation_fitness[-2]) < 0:
            cnt += 1


  0%|          | 0/20 [00:00<?, ?it/s]

{'bucketing': 28, 'encoding': 'aggregate', 'drop_act': 2, 'Decision Tree': {'max_depth': 12, 'min_samples_leaf': 64, 'criterion': 'gini'}}
bucket :  (2, 4)
After OverSampling, the shape of train_X: (34594, 9)
After OverSampling, the shape of train_y: (34594,) 

After OverSampling, counts of label '1': 17297
After OverSampling, counts of label '0': 17297
bucket :  (4, 6)
After OverSampling, the shape of train_X: (23634, 15)
After OverSampling, the shape of train_y: (23634,) 

After OverSampling, counts of label '1': 11817
After OverSampling, counts of label '0': 11817
bucket :  (6, 8)
bucket :  (8, 10)
bucket :  (10, 12)
bucket :  (12, 14)
bucket :  (14, 16)
bucket :  (16, 18)
bucket :  (18, 20)
bucket :  (20, 22)
bucket :  (22, 24)
bucket :  (24, 25)
bucket :  (25, 26)
bucket :  (26, 27)
bucket :  (27, 28)
bucket :  (28, 29)
bucket :  (29, 30)
bucket :  (30, 31)
bucket :  (31, 32)
bucket :  (32, 33)
bucket :  (33, 34)
bucket :  (34, 35)
bucket :  (35, 36)
bucket :  (36, 37)
bucket :  (

  5%|▌         | 1/20 [02:52<54:44, 172.87s/it]

{'bucketing': 27, 'encoding': 'aggregate', 'drop_act': 8, 'Decision Tree': {'max_depth': 7, 'min_samples_leaf': 47, 'criterion': 'gini'}}
bucket :  (2, 4)
After OverSampling, the shape of train_X: (34594, 8)
After OverSampling, the shape of train_y: (34594,) 

After OverSampling, counts of label '1': 17297
After OverSampling, counts of label '0': 17297
bucket :  (4, 6)
After OverSampling, the shape of train_X: (23472, 13)
After OverSampling, the shape of train_y: (23472,) 

After OverSampling, counts of label '1': 11736
After OverSampling, counts of label '0': 11736
bucket :  (6, 8)
bucket :  (8, 10)
bucket :  (10, 12)
bucket :  (12, 14)
bucket :  (14, 16)
bucket :  (16, 18)
bucket :  (18, 20)
bucket :  (20, 22)
bucket :  (22, 24)
bucket :  (24, 26)
bucket :  (26, 27)
bucket :  (27, 28)
bucket :  (28, 29)
bucket :  (29, 30)
bucket :  (30, 31)
bucket :  (31, 32)
bucket :  (32, 33)
bucket :  (33, 34)
bucket :  (34, 35)
bucket :  (35, 36)
bucket :  (36, 37)
bucket :  (37, 38)
bucket :  (3

 10%|█         | 2/20 [05:40<50:58, 169.91s/it]

...

 90%|█████████ | 18/20 [53:46<06:07, 183.70s/it]

{'bucketing': 15, 'encoding': 'index', 'drop_act': 6, 'Xgboost': {'max_depth': 14, 'n_estimators': 128, 'learning_rate': 0.05}}
bucket :  (2, 5)
After OverSampling, the shape of train_X: (46950, 14)
After OverSampling, the shape of train_y: (46950,) 

After OverSampling, counts of label '1': 23475
After OverSampling, counts of label '0': 23475
bucket :  (5, 8)
After OverSampling, the shape of train_X: (32620, 38)
After OverSampling, the shape of train_y: (32620,) 

After OverSampling, counts of label '1': 16310
After OverSampling, counts of label '0': 16310
bucket :  (8, 11)
bucket :  (11, 14)
bucket :  (14, 17)
bucket :  (17, 20)
bucket :  (20, 23)
bucket :  (23, 26)
bucket :  (26, 29)
bucket :  (29, 31)
bucket :  (31, 33)
bucket :  (33, 35)
bucket :  (35, 37)
bucket :  (37, 39)
bucket :  (39, 41)


 95%|█████████▌| 19/20 [56:49<03:03, 183.47s/it]

{'bucketing': 15, 'encoding': 'index', 'drop_act': 6, 'Xgboost': {'max_depth': 14, 'n_estimators': 128, 'learning_rate': 0.05}}
bucket :  (2, 5)
After OverSampling, the shape of train_X: (46950, 14)
After OverSampling, the shape of train_y: (46950,) 

After OverSampling, counts of label '1': 23475
After OverSampling, counts of label '0': 23475
bucket :  (5, 8)
After OverSampling, the shape of train_X: (32620, 38)
After OverSampling, the shape of train_y: (32620,) 

After OverSampling, counts of label '1': 16310
After OverSampling, counts of label '0': 16310
bucket :  (8, 11)
bucket :  (11, 14)
bucket :  (14, 17)
bucket :  (17, 20)
bucket :  (20, 23)
bucket :  (23, 26)
bucket :  (26, 29)
bucket :  (29, 31)
bucket :  (31, 33)
bucket :  (33, 35)
bucket :  (35, 37)
bucket :  (37, 39)
bucket :  (39, 41)


100%|██████████| 20/20 [59:51<00:00, 179.58s/it]


In [13]:
# 849분 (14시간)

print(generation_fitness)
print(sum(generation_time))
print(generation_score)
print(generation_failure_rate)
print(best_pop[-1])
print(tot_score[np.argmax(fitness)])

[0.7255, 0.6965, 0.7175, 0.6805, 0.694, 0.7165, 0.7725, 0.7305, 0.721, 0.728, 0.684, 0.765, 0.733, 0.8055]
50979.18000000001
[0.8175, 0.804, 0.8015, 0.8035, 0.8205, 0.849, 0.895, 0.888, 0.8775, 0.8715, 0.838, 0.903, 0.929, 0.9585]
[0.0181, 0.0265, 0.0724, 0.0224, 0.0307, 0.0384, 0.0259, 0.0058, 0.0071, 0.0038, 0.0086, 0.0086, 0.0, 0.0]
{'bucketing': 15, 'encoding': 'index', 'drop_act': 8, 'LightGBM': {'max_depth': 10, 'num_leaves': 325, 'min_child_samples': 5}}
0.98


# Experiment 2

### - Genetic Algorithm

In [35]:
"""
GA : Genetic Algorithm
RS : Random Search
"""
# phase = 1 # drop_activity(GA) + encoding(GA) + bucketing(GA) + ML params(GA) + hyperparameter opt(GA)
phase = 2 # drop_activity(GA) + encoding(GA) + bucketing(GA) + ML params(GA) + hyperparameter opt(RS)
# phase = 3 # drop_activity(GA) + encoding(GA) + bucketing(GA) + ML params(RS) + hyperparameter opt(RS)
# phase = 4 # drop_activity(RS) + encoding(RS) + bucketing(RS) + ML params(RS) + hyperparameter opt(RS)

GA = Genetic_Algorithm(df, options, combi, caseid, activity, ts, label, other_features, phase)
GA.feature_type()


In [36]:
cnt = 0

best_pop = []

populations = GA.initial_populations(5)


generation_fitness = []
generation_score = []
generation_time = []
generation_failure_rate = []
for n_iter in range(max_iter):
    new_population = []
    tot_score = []
    tot_time = []
    failure_rate = []
    for pop in tqdm(populations):
        start_time = time.time()
        new_population.append(pop)
        print(pop)
        if pop['drop_act'] == 2:
            df_list = df_list1
        elif pop['drop_act'] == 4:
            df_list = df_list2
        elif pop['drop_act'] == 6:
            df_list = df_list3
        else:
            df_list = df_list4

        if pop['encoding'] == 'last_state':
            if pop['bucketing'] < 5:
                pop['bucketing'] = 5
            bound_list = prefix_bound(pop['bucketing'], drop_low=True)
        else:
            bound_list = prefix_bound(pop['bucketing'])

        score = []
        for bounds in bound_list:
            lower, upper = bounds
            merge_df = pd.DataFrame()
            print('bucket : ',(lower, upper))
            for idx in range(lower, upper):
                prefix_df = df_list[idx-2]
                prefix_df = GA.encoding(prefix_df, pop['encoding'], idx)
                merge_df = pd.concat([merge_df, prefix_df], sort=False)

            merge_df = merge_df.fillna(0)

            X_train, y_train, X_test, y_test = GA.train_test_set_split(merge_df, pop['encoding'])    

            # if 'Decision Tree' in pop:
            #     score.append(GA.decision_tree(pop['Decision Tree'], X_train, y_train, X_test, y_test))
            # elif 'Random Forest' in pop:
            #     score.append(GA.random_forest(pop['Random Forest'], X_train, y_train, X_test, y_test))
            # elif 'LightGBM' in pop:
            #     score.append(GA.light_gbm(pop['LightGBM'], X_train, y_train, X_test, y_test))
            # else: 
            #     score.append(GA.xgboost(pop['Xgboost'], X_train, y_train, X_test, y_test))                    
                
            start = time.time()
            if pop['models'] == 'Decision Tree':
                model = DecisionTreeClassifier()
                model.fit(X_train, y_train)
                score.append(round((accuracy_score(y_test, model.predict(X_test))+roc_auc_score(y_test, model.predict_proba(X_test)[:, 1]))/2, 4))
                
            elif pop['models'] == 'Random Forest':
                model = RandomForestClassifier()
                model.fit(X_train, y_train)
                score.append(round((accuracy_score(y_test, model.predict(X_test))+roc_auc_score(y_test, model.predict_proba(X_test)[:, 1]))/2, 4))
            elif pop['models'] == 'LightGBM':
                model = LGBMClassifier()
                model.fit(X_train, y_train)
                score.append(round((accuracy_score(y_test, model.predict(X_test))+roc_auc_score(y_test, model.predict_proba(X_test)[:, 1]))/2, 4))
            else: 
                model = XGBClassifier()
                model.fit(X_train, y_train)
                score.append(round((accuracy_score(y_test, model.predict(X_test))+roc_auc_score(y_test, model.predict_proba(X_test)[:, 1]))/2, 4))
    
        end_time = time.time()

        elapsed_time = round(end_time-start_time,4)

        tot_time.append(elapsed_time)
        tot_score.append(round(sum(score)/len(score),4))

        min_proba = 0.7
        failure_rate.append(len([i for i in score if i < min_proba])/len(score))

    fitness = GA.fitness(tot_score, failure_rate, tot_time)
    best_pop.append(populations[np.argmax(fitness)])

    N = len(new_population)
    pop1 = GA.select_population(new_population, fitness, N*sr)
    pop2 = GA.crossover(new_population, fitness, N, cr)
    pop3 = GA.mutation(new_population, N, mr)

    populations = pop1 + pop2 + pop3

    generation_fitness.append(round(sum(fitness)/len(fitness),4))
    generation_time.append(sum(tot_time))
    generation_score.append(round(sum(tot_score)/len(tot_score),4))
    generation_failure_rate.append(round(sum(failure_rate)/len(failure_rate),4))
    
    if cnt > 5:
        break
    
    elif len(generation_fitness) > 1:
        if abs(generation_fitness[-1]-generation_fitness[-2]) < 0.001:
            break 
        elif (generation_fitness[-1]-generation_fitness[-2]) < 0:
            cnt += 1
            

  0%|          | 0/20 [00:00<?, ?it/s]

{'bucketing': 1, 'encoding': 'index', 'drop_act': 6, 'models': 'Decision Tree', 'Decision Tree': {'max_depth': 9, 'min_samples_leaf': 42, 'criterion': 'gini'}}
bucket :  (2, 40)


  5%|▌         | 1/20 [02:24<45:45, 144.47s/it]

{'bucketing': 13, 'encoding': 'aggregate', 'drop_act': 6, 'models': 'Decision Tree', 'Decision Tree': {'max_depth': 15, 'min_samples_leaf': 29, 'criterion': 'entropy'}}
bucket :  (2, 5)
After OverSampling, the shape of train_X: (46950, 10)
After OverSampling, the shape of train_y: (46950,) 

After OverSampling, counts of label '1': 23475
After OverSampling, counts of label '0': 23475
bucket :  (5, 8)
After OverSampling, the shape of train_X: (32620, 17)
After OverSampling, the shape of train_y: (32620,) 

After OverSampling, counts of label '1': 16310
After OverSampling, counts of label '0': 16310
bucket :  (8, 11)
bucket :  (11, 14)
bucket :  (14, 17)
bucket :  (17, 20)
bucket :  (20, 23)
bucket :  (23, 26)
bucket :  (26, 29)
bucket :  (29, 32)
bucket :  (32, 35)
bucket :  (35, 38)
bucket :  (38, 41)


 10%|█         | 2/20 [05:15<47:59, 159.95s/it]

...

 90%|█████████ | 18/20 [40:18<04:28, 134.28s/it]

{'bucketing': 12, 'encoding': 'index', 'drop_act': 2, 'models': 'Decision Tree', 'Decision Tree': {'max_depth': 15, 'min_samples_leaf': 29, 'criterion': 'entropy'}}
bucket :  (2, 6)
After OverSampling, the shape of train_X: (59022, 25)
After OverSampling, the shape of train_y: (59022,) 

After OverSampling, counts of label '1': 29511
After OverSampling, counts of label '0': 29511
bucket :  (6, 10)
bucket :  (10, 14)
bucket :  (14, 17)
bucket :  (17, 20)
bucket :  (20, 23)
bucket :  (23, 26)
bucket :  (26, 29)
bucket :  (29, 32)
bucket :  (32, 35)
bucket :  (35, 38)
bucket :  (38, 41)


 95%|█████████▌| 19/20 [42:32<02:14, 134.39s/it]

{'bucketing': 12, 'encoding': 'index', 'drop_act': 2, 'models': 'Decision Tree', 'LightGBM': {'max_depth': 9, 'num_leaves': 326, 'min_child_samples': 3}}
bucket :  (2, 6)
After OverSampling, the shape of train_X: (59022, 25)
After OverSampling, the shape of train_y: (59022,) 

After OverSampling, counts of label '1': 29511
After OverSampling, counts of label '0': 29511
bucket :  (6, 10)
bucket :  (10, 14)
bucket :  (14, 17)
bucket :  (17, 20)
bucket :  (20, 23)
bucket :  (23, 26)
bucket :  (26, 29)
bucket :  (29, 32)
bucket :  (32, 35)
bucket :  (35, 38)
bucket :  (38, 41)


100%|██████████| 20/20 [44:47<00:00, 134.36s/it]


In [37]:
print(generation_fitness)
print(sum(generation_time))
print(generation_score)
print(generation_failure_rate)
print(best_pop[-1])
print(tot_score[np.argmax(fitness)])

[0.682, 0.639, 0.691, 0.7905, 0.8485, 0.8565, 0.861, 0.8335, 0.8345, 0.8975, 0.8725, 0.941, 0.946, 0.778, 0.7715, 0.7345, 0.814]
48350.143899999995
[0.8354, 0.8052, 0.835, 0.8861, 0.924, 0.9309, 0.9409, 0.9179, 0.9125, 0.9459, 0.9687, 0.9771, 0.9797, 0.9803, 0.9803, 0.9804, 0.9805]
[0.09, 0.2184, 0.1215, 0.0528, 0.0262, 0.0143, 0.0042, 0.0143, 0.0256, 0.0167, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
{'bucketing': 12, 'encoding': 'index', 'drop_act': 2, 'models': 'Decision Tree', 'LightGBM': {'max_depth': 9, 'num_leaves': 326, 'min_child_samples': 3}}
0.9808


### - Random Search

In [39]:
from sklearn.model_selection import RandomizedSearchCV

In [40]:
model = {'Decision Tree' : DecisionTreeClassifier(), 'Random Forest' : RandomForestClassifier(), 'LightGBM' : LGBMClassifier(), 'Xgboost' : XGBClassifier()}

start_time = time.time()

pop = best_pop[-1]

best_params = []

if pop['drop_act'] == 2:
    df_list = df_list1
elif pop['drop_act'] == 4:
    df_list = df_list2
elif pop['drop_act'] == 6:
    df_list = df_list3
else:
    df_list = df_list4

bound_list = prefix_bound(pop['bucketing'])

score = []
for bounds in bound_list:
    lower, upper = bounds
    merge_df = pd.DataFrame()
    print('bucket : ',(lower, upper))
    for idx in range(lower, upper):
        prefix_df = df_list[idx-2]
        prefix_df = GA.encoding(prefix_df, pop['encoding'], idx)
        merge_df = pd.concat([merge_df, prefix_df], sort=False)

    merge_df = merge_df.fillna(0)

    X_train, y_train, X_test, y_test = GA.train_test_set_split(merge_df, pop['encoding']) 

    space = options['params'][pop['models']]

    search = RandomizedSearchCV(RandomForestClassifier(), space, n_iter=20, scoring='roc_auc', n_jobs=-1, random_state=1)
    result = search.fit(X_train, y_train)
    random_search_params = result.best_params_
    best_params.append(random_search_params)

    print('Best Score: %s' % result.best_score_)
    print('Best Hyperparameters: %s' % result.best_params_)
    
    model = RandomForestClassifier(**random_search_params, random_state = 42)
    model.fit(X_train, y_train)
    score.append(round((accuracy_score(y_test, model.predict(X_test))+roc_auc_score(y_test, model.predict_proba(X_test)[:, 1]))/2, 4))


end_time = time.time()
elapsed_time = round(end_time-start_time,4)

print(elapsed_time)
print(score)

bucket :  (2, 6)
After OverSampling, the shape of train_X: (59022, 25)
After OverSampling, the shape of train_y: (59022,) 

After OverSampling, counts of label '1': 29511
After OverSampling, counts of label '0': 29511


C:\Users\yeon1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 8 is smaller than n_iter=20. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Score: 0.902338653407735
Best Hyperparameters: {'min_samples_leaf': 5, 'max_depth': 2, 'criterion': 'gini'}
bucket :  (6, 10)


C:\Users\yeon1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 8 is smaller than n_iter=20. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Score: 0.8369378673875927
Best Hyperparameters: {'min_samples_leaf': 100, 'max_depth': 20, 'criterion': 'gini'}
bucket :  (10, 14)


...

Best Score: 0.953215551675554
Best Hyperparameters: {'min_samples_leaf': 5, 'max_depth': 20, 'criterion': 'gini'}
bucket :  (35, 38)


C:\Users\yeon1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 8 is smaller than n_iter=20. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Score: 0.9577729304580089
Best Hyperparameters: {'min_samples_leaf': 5, 'max_depth': 20, 'criterion': 'entropy'}
bucket :  (38, 41)


C:\Users\yeon1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 8 is smaller than n_iter=20. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Score: 0.9644813956521302
Best Hyperparameters: {'min_samples_leaf': 5, 'max_depth': 20, 'criterion': 'entropy'}
275.4128
[0.7996, 0.7815, 0.868, 0.8894, 0.9012, 0.9017, 0.9101, 0.9274, 0.9398, 0.9434, 0.9403, 0.9473]


# Experiment 3
### - Genetic Algorithm

In [41]:
"""
GA : Genetic Algorithm
RS : Random Search
"""
# phase = 1 # drop_activity(GA) + encoding(GA) + bucketing(GA) + ML params(GA) + hyperparameter opt(GA)
# phase = 2 # drop_activity(GA) + encoding(GA) + bucketing(GA) + ML params(GA) + hyperparameter opt(RS)
phase = 3 # drop_activity(GA) + encoding(GA) + bucketing(GA) + ML params(RS) + hyperparameter opt(RS)
# phase = 4 # drop_activity(RS) + encoding(RS) + bucketing(RS) + ML params(RS) + hyperparameter opt(RS)

GA = Genetic_Algorithm(df, options, combi, caseid, activity, ts, label, other_features, phase)
GA.feature_type()


In [43]:
populations = GA.initial_populations(20)

best_pop = []
generation_fitness = []
generation_score = []
generation_time = []
generation_failure_rate = []
for n_iter in range(max_iter):
    new_population = []
    tot_score = []
    tot_time = []
    failure_rate = []
    for pop in tqdm(populations):
        new_population.append(pop)
        start_time = time.time()
        print(pop)
        if pop['drop_act'] == 2:
            df_list = df_list1
        elif pop['drop_act'] == 4:
            df_list = df_list2
        elif pop['drop_act'] == 6:
            df_list = df_list3
        else:
            df_list = df_list4
            
        if pop['encoding'] == 'last_state':
            bound_list = prefix_bound(pop['bucketing'], drop_low=True)
        else:
            bound_list = prefix_bound(pop['bucketing'])

        score = []
        for bounds in bound_list:
            lower, upper = bounds
            merge_df = pd.DataFrame()
            print('bucket : ',(lower, upper))
            for idx in range(lower, upper):
                prefix_df = df_list[idx-2]
                prefix_df = GA.encoding(prefix_df, pop['encoding'], idx)
                merge_df = pd.concat([merge_df, prefix_df], sort=False)

            merge_df = merge_df.fillna(0)

            X_train, y_train, X_test, y_test = GA.train_test_set_split(merge_df, pop['encoding'])    

            #start = time.time()
            model = XGBClassifier()
            model.fit(X_train, y_train)
            score.append(round((accuracy_score(y_test, model.predict(X_test))+roc_auc_score(y_test, model.predict_proba(X_test)[:, 1]))/2, 4))
            

        end_time = time.time()

        elapsed_time = round(end_time-start_time,2)
        tot_time.append(elapsed_time)
        tot_score.append(round(sum(score)/len(score),2))

        min_proba = 0.7
        failure_rate.append(len([i for i in score if i < min_proba])/len(score))

    fitness = GA.fitness(tot_score, failure_rate, tot_time)
    best_pop.append(populations[np.argmax(fitness)])

    N = len(new_population)
    pop1 = GA.select_population(new_population, fitness, N*sr)
    pop2 = GA.crossover(new_population, fitness, N, cr)
    pop3 = GA.mutation(new_population, N, mr)
    
    populations = pop1 + pop2 + pop3
    
    generation_fitness.append(round(sum(fitness)/len(fitness),2))
    generation_time.append(sum(tot_time))
    generation_score.append(round(sum(tot_score)/len(tot_score),2))
    generation_failure_rate.append(round(sum(failure_rate)/len(failure_rate),2))

  0%|          | 0/80 [00:00<?, ?it/s]

{'bucketing': 33, 'encoding': 'aggregate', 'drop_act': 4, 'models': 'Decision Tree', 'Decision Tree': {'max_depth': 18, 'min_samples_leaf': 24, 'criterion': 'entropy'}}
bucket :  (2, 4)
After OverSampling, the shape of train_X: (34594, 8)
After OverSampling, the shape of train_y: (34594,) 

After OverSampling, counts of label '1': 17297
After OverSampling, counts of label '0': 17297
bucket :  (4, 6)
After OverSampling, the shape of train_X: (23472, 13)
After OverSampling, the shape of train_y: (23472,) 

After OverSampling, counts of label '1': 11736
After OverSampling, counts of label '0': 11736
bucket :  (6, 8)
bucket :  (8, 10)
bucket :  (10, 12)
bucket :  (12, 14)
bucket :  (14, 15)
bucket :  (15, 16)
bucket :  (16, 17)
bucket :  (17, 18)
bucket :  (18, 19)
bucket :  (19, 20)
bucket :  (20, 21)
bucket :  (21, 22)
bucket :  (22, 23)
bucket :  (23, 24)
bucket :  (24, 25)
bucket :  (25, 26)
bucket :  (26, 27)
bucket :  (27, 28)
bucket :  (28, 29)
bucket :  (29, 30)
bucket :  (30, 31)


  1%|▏         | 1/80 [03:08<4:08:13, 188.53s/it]

{'bucketing': 16, 'encoding': 'index', 'drop_act': 6, 'models': 'Decision Tree', 'Decision Tree': {'max_depth': 4, 'min_samples_leaf': 44, 'criterion': 'entropy'}}
bucket :  (2, 5)
After OverSampling, the shape of train_X: (46950, 14)
After OverSampling, the shape of train_y: (46950,) 

After OverSampling, counts of label '1': 23475
After OverSampling, counts of label '0': 23475
bucket :  (5, 8)
After OverSampling, the shape of train_X: (32620, 38)
After OverSampling, the shape of train_y: (32620,) 

After OverSampling, counts of label '1': 16310
After OverSampling, counts of label '0': 16310
bucket :  (8, 11)
bucket :  (11, 14)
bucket :  (14, 17)
bucket :  (17, 20)
bucket :  (20, 23)
bucket :  (23, 25)
bucket :  (25, 27)
bucket :  (27, 29)
bucket :  (29, 31)
bucket :  (31, 33)
bucket :  (33, 35)
bucket :  (35, 37)
bucket :  (37, 39)
bucket :  (39, 41)


  2%|▎         | 2/80 [05:44<3:40:21, 169.51s/it]

...

 98%|█████████▊| 78/80 [3:12:21<04:56, 148.12s/it]

{'bucketing': 16, 'encoding': 'index', 'drop_act': 8, 'models': 'Xgboost', 'Random Forest': {'n_estimators': 846, 'max_depth': 13, 'max_features': 'auto', 'bootstrap': False, 'criterion': 'gini'}}
bucket :  (2, 5)
After OverSampling, the shape of train_X: (46950, 14)
After OverSampling, the shape of train_y: (46950,) 

After OverSampling, counts of label '1': 23475
After OverSampling, counts of label '0': 23475
bucket :  (5, 8)
After OverSampling, the shape of train_X: (32620, 38)
After OverSampling, the shape of train_y: (32620,) 

After OverSampling, counts of label '1': 16310
After OverSampling, counts of label '0': 16310
bucket :  (8, 11)
bucket :  (11, 14)
bucket :  (14, 17)
bucket :  (17, 20)
bucket :  (20, 23)
bucket :  (23, 25)
bucket :  (25, 27)
bucket :  (27, 29)
bucket :  (29, 31)
bucket :  (31, 33)
bucket :  (33, 35)
bucket :  (35, 37)
bucket :  (37, 39)
bucket :  (39, 41)


 99%|█████████▉| 79/80 [3:14:49<02:27, 147.99s/it]

{'bucketing': 16, 'encoding': 'index', 'drop_act': 8, 'models': 'LightGBM', 'LightGBM': {'max_depth': 6, 'num_leaves': 118, 'min_child_samples': 6}}
bucket :  (2, 5)
After OverSampling, the shape of train_X: (46950, 14)
After OverSampling, the shape of train_y: (46950,) 

After OverSampling, counts of label '1': 23475
After OverSampling, counts of label '0': 23475
bucket :  (5, 8)
After OverSampling, the shape of train_X: (32620, 38)
After OverSampling, the shape of train_y: (32620,) 

After OverSampling, counts of label '1': 16310
After OverSampling, counts of label '0': 16310
bucket :  (8, 11)
bucket :  (11, 14)
bucket :  (14, 17)
bucket :  (17, 20)
bucket :  (20, 23)
bucket :  (23, 25)
bucket :  (25, 27)
bucket :  (27, 29)
bucket :  (29, 31)
bucket :  (31, 33)
bucket :  (33, 35)
bucket :  (35, 37)
bucket :  (37, 39)
bucket :  (39, 41)


100%|██████████| 80/80 [3:17:17<00:00, 147.96s/it]


In [44]:
#4104분 (68시간)
print(elapsed_time)
print(score)
print(np.mean(score))

147.79
[0.8904, 0.9166, 0.9399, 0.954, 0.9634, 0.9622, 0.9614, 0.97, 0.9736, 0.9807, 0.9833, 0.9984, 0.9967, 0.999, 0.9988, 1.0]
0.968025


### - Random Search

In [51]:
# random search
models = {'Decision Tree' : DecisionTreeClassifier(), 'Random Forest' : RandomForestClassifier(), 'LightGBM' : LGBMClassifier(), 'Xgboost' : XGBClassifier()}

start_time = time.time()

pop = best_pop[-1]

if pop['drop_act'] == 2:
    df_list = df_list1
elif pop['drop_act'] == 4:
    df_list = df_list2
elif pop['drop_act'] == 6:
    df_list = df_list3
else:
    df_list = df_list4

bound_list = prefix_bound(pop['bucketing'])

score = {'DT':[], 'RF':[], 'LGBM':[], 'XGB':[]}
best_param = {'DT':[], 'RF':[], 'LGBM':[], 'XGB':[]}
avg_score = []

for bounds in bound_list:
    lower, upper = bounds
    merge_df = pd.DataFrame()
    print('bucket : ',(lower, upper))
    for idx in range(lower, upper):
        prefix_df = df_list1[idx-2]
        prefix_df = GA.encoding(prefix_df, pop['encoding'], idx)
        merge_df = pd.concat([merge_df, prefix_df], sort=False)

    merge_df = merge_df.fillna(0)

    X_train, y_train, X_test, y_test = GA.train_test_set_split(merge_df, pop['encoding']) 
    avg = 0
    for model in models.keys():
        if model == 'Decision Tree':
            space = options['params']['Decision Tree']
            search = RandomizedSearchCV(models['Decision Tree'], space, n_iter=20, scoring='roc_auc', n_jobs=-1, random_state=1)
            result = search.fit(X_train, y_train)
            random_search_params = result.best_params_
            best_param['DT'].append(random_search_params)
            model = DecisionTreeClassifier(**random_search_params, random_state = 42)
            model.fit(X_train, y_train)
            sc = round((accuracy_score(y_test, model.predict(X_test))+roc_auc_score(y_test, model.predict_proba(X_test)[:, 1]))/2, 4)
            score['DT'].append(sc)
            avg += sc
        elif model == 'Random Forest':
            space = options['params']['Random Forest']
            search = RandomizedSearchCV(models['Random Forest'], space, n_iter=20, scoring='roc_auc', n_jobs=-1, random_state=1)
            result = search.fit(X_train, y_train)
            random_search_params = result.best_params_
            best_param['RF'].append(random_search_params)
            model = RandomForestClassifier(**random_search_params, random_state = 42)
            model.fit(X_train, y_train)
            sc = round((accuracy_score(y_test, model.predict(X_test))+roc_auc_score(y_test, model.predict_proba(X_test)[:, 1]))/2, 4)            
            score['RF'].append(sc)
            avg += sc
        elif model == 'LightGBM':
            space = options['params']['LightGBM']
            search = RandomizedSearchCV(models['LightGBM'], space, n_iter=20, scoring='roc_auc', n_jobs=-1, random_state=1)
            result = search.fit(X_train, y_train)
            random_search_params = result.best_params_
            best_param['LGBM'].append(random_search_params)
            model = LGBMClassifier(**random_search_params, random_state = 42)
            model.fit(X_train, y_train)
            sc = round((accuracy_score(y_test, model.predict(X_test))+roc_auc_score(y_test, model.predict_proba(X_test)[:, 1]))/2, 4)
            score['LGBM'].append(sc)
            avg += sc
        elif model == 'Xgboost':
            space = options['params']['Xgboost']
            search = RandomizedSearchCV(models['Xgboost'], space, n_iter=20, scoring='roc_auc', n_jobs=-1, random_state=1)
            result = search.fit(X_train, y_train)
            random_search_params = result.best_params_
            best_param['XGB'].append(random_search_params)
            model = XGBClassifier(**random_search_params, random_state = 42)
            model.fit(X_train, y_train)
            sc = round((accuracy_score(y_test, model.predict(X_test))+roc_auc_score(y_test, model.predict_proba(X_test)[:, 1]))/2, 4)
            score['XGB'].append(sc)
            avg += sc
            
    avg_score.append(round(avg/4, 4))

end_time = time.time()
elapsed_time = round(end_time-start_time,4)

bucket :  (2, 7)
After OverSampling, the shape of train_X: (70822, 36)
After OverSampling, the shape of train_y: (70822,) 

After OverSampling, counts of label '1': 35411
After OverSampling, counts of label '0': 35411


C:\Users\yeon1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 8 is smaller than n_iter=20. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
C:\Users\yeon1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\yeon1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been 

In [50]:
# import numpy as np

print(elapsed_time)
print(score)
# print(np.mean(score))

11206.9213
{'DT': [0.7794, 0.9481, 0.9379, 0.9237, 0.8919, 0.9449, 0.9414, 0.9024, 0.9541], 'RF': [0.8066, 0.9904, 0.9984, 0.9989, 0.9994, 1.0, 0.9991, 0.9982, 0.998], 'LGBM': [0.82, 0.9997, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 'XGB': [0.9985, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}


# Experiment 4
### - Random search

In [10]:
"""
GA : Genetic Algorithm
RS : Random Search
"""
# phase = 1 # drop_activity(GA) + encoding(GA) + bucketing(GA) + ML params(GA) + hyperparameter opt(GA)
# phase = 2 # drop_activity(GA) + encoding(GA) + bucketing(GA) + ML params(GA) + hyperparameter opt(RS)
# phase = 3 # drop_activity(GA) + encoding(GA) + bucketing(GA) + ML params(RS) + hyperparameter opt(RS)
phase = 4 # drop_activity(RS) + encoding(RS) + bucketing(RS) + ML params(RS) + hyperparameter opt(RS)

GA = Genetic_Algorithm(df, options, combi, caseid, activity, ts, label, other_features, phase)
GA.feature_type()

In [11]:
score = []

start_time = time.time()

populations = GA.initial_populations(100)

for pop in tqdm(populations):
    print(pop)
    if pop['drop_act'] == 2:
        df_list = df_list1
    elif pop['drop_act'] == 4:
        df_list = df_list2
    elif pop['drop_act'] == 6:
        df_list = df_list3
    else:
        df_list = df_list4

    bound_list = prefix_bound(pop['bucketing'])

    for bounds in bound_list:
        lower, upper = bounds
        merge_df = pd.DataFrame()
        print('bucket : ',(lower, upper))
        for idx in range(lower, upper):
            prefix_df = df_list[idx-2]
            prefix_df = GA.encoding(prefix_df, pop['encoding'], idx)
            merge_df = pd.concat([merge_df, prefix_df], sort=False)

        merge_df = merge_df.fillna(0)

        X_train, y_train, X_test, y_test = GA.train_test_set_split(merge_df, pop['encoding'])

        #start = time.time()
        if 'Decision Tree' in pop:
            score.append(GA.decision_tree(pop['Decision Tree'], X_train, y_train, X_test, y_test))
        elif 'Random Forest' in pop:
            score.append(GA.random_forest(pop['Random Forest'], X_train, y_train, X_test, y_test))
        elif 'LightGBM' in pop:
            score.append(GA.light_gbm(pop['LightGBM'], X_train, y_train, X_test, y_test))
        else: 
            score.append(GA.xgboost(pop['Xgboost'], X_train, y_train, X_test, y_test))


  0%|          | 0/400 [00:00<?, ?it/s]

{'bucketing': 4, 'encoding': 'aggregate', 'drop_act': 2, 'models': 'Decision Tree', 'Decision Tree': {'max_depth': 16, 'min_samples_leaf': 69, 'criterion': 'gini'}}
bucket :  (2, 12)
After OverSampling, the shape of train_X: (112708, 22)
After OverSampling, the shape of train_y: (112708,) 

After OverSampling, counts of label '1': 56354
After OverSampling, counts of label '0': 56354
bucket :  (12, 22)
bucket :  (22, 32)
bucket :  (32, 41)


  0%|          | 1/400 [02:55<19:28:23, 175.70s/it]

{'bucketing': 4, 'encoding': 'aggregate', 'drop_act': 6, 'models': 'Decision Tree', 'Decision Tree': {'max_depth': 13, 'min_samples_leaf': 47, 'criterion': 'gini'}}
bucket :  (2, 12)
After OverSampling, the shape of train_X: (112244, 20)
After OverSampling, the shape of train_y: (112244,) 

After OverSampling, counts of label '1': 56122
After OverSampling, counts of label '0': 56122
bucket :  (12, 22)
bucket :  (22, 32)
bucket :  (32, 41)


  0%|          | 2/400 [05:50<19:21:53, 175.16s/it]

...

{'bucketing': 31, 'encoding': 'aggregate', 'drop_act': 8, 'models': 'Xgboost', 'Xgboost': {'max_depth': 14, 'n_estimators': 652, 'learning_rate': 0.1}}
bucket :  (2, 4)
After OverSampling, the shape of train_X: (34594, 8)
After OverSampling, the shape of train_y: (34594,) 

After OverSampling, counts of label '1': 17297
After OverSampling, counts of label '0': 17297
bucket :  (4, 6)
After OverSampling, the shape of train_X: (23472, 13)
After OverSampling, the shape of train_y: (23472,) 

After OverSampling, counts of label '1': 11736
After OverSampling, counts of label '0': 11736
bucket :  (6, 8)
bucket :  (8, 10)
bucket :  (10, 12)
bucket :  (12, 14)
bucket :  (14, 16)
bucket :  (16, 18)
bucket :  (18, 19)
bucket :  (19, 20)
bucket :  (20, 21)
bucket :  (21, 22)
bucket :  (22, 23)
bucket :  (23, 24)
bucket :  (24, 25)
bucket :  (25, 26)
bucket :  (26, 27)
bucket :  (27, 28)
bucket :  (28, 29)
bucket :  (29, 30)
bucket :  (30, 31)
bucket :  (31, 32)
bucket :  (32, 33)
bucket :  (33, 34

100%|█████████▉| 399/400 [22:20:09<04:18, 258.95s/it]

{'bucketing': 12, 'encoding': 'aggregate', 'drop_act': 2, 'models': 'Xgboost', 'Xgboost': {'max_depth': 5, 'n_estimators': 566, 'learning_rate': 0.01}}
bucket :  (2, 6)
After OverSampling, the shape of train_X: (59022, 15)
After OverSampling, the shape of train_y: (59022,) 

After OverSampling, counts of label '1': 29511
After OverSampling, counts of label '0': 29511
bucket :  (6, 10)
bucket :  (10, 14)
bucket :  (14, 17)
bucket :  (17, 20)
bucket :  (20, 23)
bucket :  (23, 26)
bucket :  (26, 29)
bucket :  (29, 32)
bucket :  (32, 35)
bucket :  (35, 38)
bucket :  (38, 41)


100%|██████████| 400/400 [22:23:26<00:00, 201.52s/it]


IndexError: list index out of range

In [13]:
end_time = time.time()
tot_time = round(end_time-start_time,4)
tot_score = (round(sum(score)/len(score),4))

# best_pop = populations[np.argmax(score)]
best_score = max(score)

# print('best instance is ', best_pop)        
print('best score is ', best_score)        
print('total time is ', tot_time)

best score is  1.0
total time is  96289.6976
